In [11]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('reviews.csv')

In [4]:
def tokenizer(text):
  text = re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
      ' '.join(emoticons).replace('-', '')
  return text


In [8]:
X = []
sentences = list(df['reviews'])
for sen in sentences:
    X.append(tokenizer(sen))
y = df['sentiment']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)